**Werkwijze:**

1️⃣ Eerst heb ik geïdentificeerd welke pagina’s afbeeldingen bevatten en deze opgeslagen in een lijst.

2️⃣ Op iedere pagina heb ik via `requests` de HTML-bron opgehaald en met `BeautifulSoup` alle `<img>`-tags uitgelezen.

3️⃣ Omdat de site **thumbnail-plaatjes** gebruikt, waarbij de originele afbeelding pas zichtbaar wordt wanneer je `thmbnl` uit de bestandsnaam haalt, heb ik een omzet-functie gemaakt die automatisch de juiste full-resolution URL afleidt.

4️⃣ Sommige thumbnails hadden géén full-image op de server → in die gevallen heb ik een **fallback** toegevoegd die de thumbnail zelf opslaat.

5️⃣ De site bevat ook veel **niet-relevante afbeeldingen** zoals spacers, advertenties, logo’s en achtergronden. Deze heb ik gefilterd op basis van sleutelwoorden (bv. `spacer`, `logo`, `banner`).

6️⃣ Alle afbeeldingen zijn vervolgens in **één map** opgeslagen, waarbij bestandsnamen automatisch werden opgeschoond (bijv. `"thmbnl_portrait.jpg"` → `"portrait.jpg"`).

7️⃣ Tot slot print het script een **samenvatting** met het aantal gevonden thumbnails, opgeslagen full-images en fallback-thumbnails.


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import os.path as op


# te scrapen pagina's
PAGES = [
    "https://www.rembrandtpainting.net/rembrandt_painting_1626-35.htm",
    "https://www.rembrandtpainting.net/rembrandt_painting_1636_54.htm",
    "https://www.rembrandtpainting.net/rembrandt_painting_1655-69.htm",
    "https://www.rembrandtpainting.net/rembrandt_self_portraits.htm",
    "https://www.rembrandtpainting.net/rembrandt_drawings_start.htm",
    "https://www.rembrandtpainting.net/rmbrdnt_selected_etchings/rembrandt_etchings.htm",
    "https://www.rembrandtpainting.net/rembrandt's_prodigal_son.html",
    "https://www.rembrandtpainting.net/famous_works.html",
    "https://www.rembrandtpainting.net/rembrandt_books.htm",
]

# output-directory
SAVE_DIR = "rembrandt_images"
os.makedirs(SAVE_DIR, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; RembrandtScraper/1.0)"
}

# Bestandsnaam-patronen om te skippen (UI / layout / ads)
BAD_KEYWORDS = [
    "spacer", "background", "backgrnd", "poster", "dvd", "sign",
    "ad_", "signature", "artcom", "sfumatura", "logo", "banner"
]

VALID_EXTS = (".jpg", ".jpeg", ".png", ".gif")



# HULPFUNCTIES

def build_full_candidates(thumb_url: str) -> list:
    """
    Genereer mogelijke full-size URL's op basis van één thumbnail-URL.
    """
    candidates = []

    # '/thmbnls/' -> '/' + 'thmbnl' overal uit bestandsnaam
    v1 = thumb_url.replace("/thmbnls/", "/")
    v1 = v1.replace("thmbnl_", "").replace("_thmbnl", "").replace("thmbnl", "")
    candidates.append(v1)

    # Alleen 'thmbnl' uit bestandsnaam, map ongewijzigd
    v2 = thumb_url.replace("thmbnl_", "").replace("_thmbnl", "").replace("thmbnl", "")
    if v2 != v1:
        candidates.append(v2)

    return candidates


def clean_filename_from_url(url: str) -> str:
    """
    Haal alleen de bestandsnaam uit de URL en verwijder 'thmbnl' varianten.
    Zorg dat er een geldige extensie op zit.
    """
    name = op.basename(url)

    # 'thmbnl' uit de naam slopen
    name = name.replace("thmbnl_", "").replace("_thmbnl", "").replace("thmbnl", "")

    # Als er geen extensie is, of geen geldige, plak .jpg eraan
    if not any(name.lower().endswith(ext) for ext in VALID_EXTS):
        name += ".jpg"

    return name


def scrape_page(page_url: str) -> dict:
    """
    Scrape één pagina. Alles wordt in SAVE_DIR gezet.
    Retourneert statistieken.
    """
    print("\n=======================================")
    print(f"[PAGE] Start scrape: {page_url}")
    print("=======================================\n")

    # HTML ophalen
    resp = requests.get(page_url, headers=HEADERS)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    img_tags = soup.find_all("img")
    print(f"[INFO] Gevonden <img>-tags op deze pagina: {len(img_tags)}")

    # Counters voor deze pagina
    count_full = 0
    count_thumb_fallback = 0
    count_skipped_layout = 0
    count_thumbnails_seen = 0

    for img in img_tags:
        src = img.get("src")
        if not src:
            continue

        lower_src = src.lower()

        # layout / advertentie / UI images overslaan
        if any(bad in lower_src for bad in BAD_KEYWORDS):
            count_skipped_layout += 1
            print(f"[SKIP] Layout/advert image: {src}")
            continue

        # alleen thumbnails met 'thmbnl' in de naam
        if "thmbnl" not in lower_src:
            continue

        count_thumbnails_seen += 1

        thumb_url = urljoin(page_url, src)
        print("\n[IMG] Thumbnail:", thumb_url)

        full_candidates = build_full_candidates(thumb_url)
        chosen_full = None
        img_bytes = None

        # probeer full-size kandidaten
        for cand in full_candidates:
            try:
                r = requests.get(cand, headers=HEADERS, timeout=15)
                if r.status_code == 200 and len(r.content) > 500:
                    chosen_full = cand
                    img_bytes = r.content
                    print(f"[OK] Full gevonden: {cand}")
                    break
                else:
                    print(f"[INFO] Kandidaat faalt (status={r.status_code}, size={len(r.content)}): {cand}")
            except Exception as e:
                print(f"[ERROR] Fout bij kandidaat {cand}: {e}")

        # opslaan: full of thumbnail
        if chosen_full and img_bytes:
            filename = clean_filename_from_url(chosen_full)
            save_path = op.join(SAVE_DIR, filename)

            with open(save_path, "wb") as f:
                f.write(img_bytes)

            count_full += 1
            print(f"[SAVE] Full-size opgeslagen als: {save_path}")
        else:
            print("[WAARSCHUWING] Geen bruikbare full-size, thumbnail als fallback.")
            try:
                r_thumb = requests.get(thumb_url, headers=HEADERS, timeout=15)
                filename = clean_filename_from_url(thumb_url)
                save_path = op.join(SAVE_DIR, filename)

                with open(save_path, "wb") as f:
                    f.write(r_thumb.content)

                count_thumb_fallback += 1
                print(f"[SAVE] Thumbnail opgeslagen als: {save_path}")
            except Exception as e:
                print(f"[ERROR] Kon thumbnail ook niet opslaan: {e}")

    total_saved = count_full + count_thumb_fallback

    # Samenvatting voor deze pagina
    print("\n----------- PAGINA SAMENVATTING -----------")
    print(f"Pagina                       : {page_url}")
    print(f"Gezién thumbnails (thmbnl)   : {count_thumbnails_seen}")
    print(f"Overgeslagen layout/ads      : {count_skipped_layout}")
    print(f"Full-size opgeslagen         : {count_full}")
    print(f"Thumbnail-fallbacks          : {count_thumb_fallback}")
    print(f"TOTAAL opgeslagen (deze pag) : {total_saved}")
    print("-------------------------------------------\n")

    return {
        "page": page_url,
        "thumbs_seen": count_thumbnails_seen,
        "skipped_layout": count_skipped_layout,
        "full": count_full,
        "thumb_fallback": count_thumb_fallback,
        "total_saved": total_saved,
    }


# scraping van de pagina's
all_stats = []

for url in PAGES:
    stats = scrape_page(url)
    all_stats.append(stats)

# Globale samenvatting
global_full = sum(s["full"] for s in all_stats)
global_thumb_fallback = sum(s["thumb_fallback"] for s in all_stats)
global_total = sum(s["total_saved"] for s in all_stats)
global_thumbs_seen = sum(s["thumbs_seen"] for s in all_stats)
global_skipped = sum(s["skipped_layout"] for s in all_stats)

print("\n=======================================")
print("           GLOBALE SAMENVATTING        ")
print("=======================================")
print(f"Totaal pagina's gescrapet        : {len(PAGES)}")
print(f"Totaal gezién thumbnails         : {global_thumbs_seen}")
print(f"Totaal overgeslagen layout/ads   : {global_skipped}")
print(f"Totaal full-size opgeslagen      : {global_full}")
print(f"Totaal thumbnail-fallbacks       : {global_thumb_fallback}")
print("---------------------------------------")
print(f"TOTAAL afbeeldingen opgeslagen   : {global_total}")
print(f"Uitvoer-map                      : {SAVE_DIR}")
print("=======================================\n")



[PAGE] Start scrape: https://www.rembrandtpainting.net/rembrandt_painting_1626-35.htm

[INFO] Gevonden <img>-tags op deze pagina: 53
[SKIP] Layout/advert image: graphics/ad_signature.jpg
[SKIP] Layout/advert image: graphics/artcomposter_c.jpg
[SKIP] Layout/advert image: graphics/ad_signature.jpg
[SKIP] Layout/advert image: graphics/dvdrembrandtsmall.jpg
[SKIP] Layout/advert image: graphics/spacerx.gif

[IMG] Thumbnail: https://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/thmbnl_musical_allegory.jpg
[OK] Full gevonden: https://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/musical_allegory.jpg
[SAVE] Full-size opgeslagen als: rembrandt_images\musical_allegory.jpg

[IMG] Thumbnail: https://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/thmbnl_history_painting.jpg
[OK] Full gevonden: https://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/history_painting.jpg
[SAVE] Full-size opgeslagen als: rembrandt_images\history_painting.jpg

[IMG] Thumbnail: 